# System-theoretic Methods

<h2>
September 19, 2023<br/>
pyMOR School 2023
</h2>

$$
\newcommand{\bbC}{\mathbb{C}}
\newcommand{\Cnn}{\bbC^{n \times n}}
\newcommand{\Cpm}{\bbC^{p \times m}}
\newcommand{\bbR}{\mathbb{R}}
\newcommand{\Rn}{\bbR^{n}}
\newcommand{\Rr}{\bbR^{r}}
\newcommand{\Rm}{\bbR^{m}}
\newcommand{\Rp}{\bbR^{p}}
\newcommand{\Rk}{\bbR^{k}}
\newcommand{\Rnn}{\bbR^{n \times n}}
\newcommand{\Rnm}{\bbR^{n \times m}}
\newcommand{\Rpn}{\bbR^{p \times n}}
\newcommand{\Rpm}{\bbR^{p \times m}}
\newcommand{\Rnr}{\bbR^{n \times r}}
\newcommand{\Rrr}{\bbR^{r \times r}}
\newcommand{\Rrm}{\bbR^{r \times m}}
\newcommand{\Rpr}{\bbR^{p \times r}}
\newcommand{\cH}{\mathcal{H}}
\newcommand{\cK}{\mathcal{K}}
\newcommand{\cL}{\mathcal{L}}
\newcommand{\cT}{\mathcal{T}}
\newcommand{\hA}{\hat{A}}
\newcommand{\hB}{\hat{B}}
\newcommand{\hC}{\hat{C}}
\newcommand{\hE}{\hat{E}}
\newcommand{\hH}{\hat{H}}
\newcommand{\hY}{\hat{Y}}
\newcommand{\hx}{\hat{x}}
\newcommand{\hy}{\hat{y}}
\newcommand{\tA}{\tilde{A}}
\newcommand{\tB}{\tilde{B}}
\newcommand{\tC}{\tilde{C}}
\newcommand{\tE}{\tilde{E}}
\newcommand{\tran}{\operatorname{T}}
\newcommand{\herm}{\operatorname{H}}
\newcommand{\Real}{\operatorname{Re}}
\newcommand{\imag}{\boldsymbol{\imath}}
\newcommand{\tr}{\operatorname{tr}}
\newcommand{\myspan}{\operatorname{span}}
\newcommand{\dif}[1]{\operatorname{d}\!{#1}}
$$

# What this actually is all about:

<center>
<img src="figures/system_fom.svg" alt="system" width="60%">
</center>

<center>
<img src="figures/mor_system_fo_v2.svg" alt="mor" width="40%">
</center>

# Outline

<h2>
1. Linear Time-Invariant (LTI) Systems<br/>
2. Transfer Function and Realizations<br/>
3. Projection-based Model order Reduction<br />
4. System Analysis<br/>
5. A Selection of MOR Methods<br/>
</h2>

# Restrictions for this lecture

- Only continuous-time systems
  - Discrete-time is treated in
    [[Antoulas '05]](https://doi.org/10.1137/1.9780898718713)
- Only unstructured systems
- No differential-algebraic systems
  - For DAE aspects see
    [[Voigt '19]](https://www.math.uni-hamburg.de/home/voigt/Modellreduktion_SoSe19/Notes_ModelReduction.pdf),
    [[Gugercin/Stykel/Wyatt '13]](https://doi.org/10.1137/130906635),
    [[Mehrmann/Stykel '05]](https://doi.org/10.1007/3-540-27909-1_3),
    [[Stykel '04]](https://doi.org/10.1007/s00498-004-0141-4)
- No non-linearities
- No parameter dependencies

# Linear Time-Invariant (LTI) Systems

## Setting for this course

### First-order State-space Systems (pyMOR: [`LTIModel`](https://docs.pymor.org/2023-1-0/autoapi/pymor/models/iosys/index.html#pymor.models.iosys.LTIModel))

$$
\begin{equation}\tag{$\Sigma$}
  \begin{aligned}
    E \dot{x}(t) & = A x(t) + B u(t), \\
    y(t) & = C x(t) + D u(t).
  \end{aligned}
\end{equation}
$$

Here

- $x(t) \in \Rn$ is called the *state*,
- $u(t) \in \Rm$ is called the *input*,
- $y(t) \in \Rp$ is called the *output*

of the LTI system.
Correspondingly, we have

$$
\begin{align*}
  E, A \in \Rnn, \qquad
  B \in \Rnm, \qquad
  C \in \Rpn, \quad\text{and}\quad
  D \in \Rpm.
\end{align*}
$$

We assume
$t \in [0, \infty)$,
$x(0) = 0$,
$E$ is invertible,
$E^{-1} A$ is Hurwitz, and
$D = 0$.

## Examples

### Heat Equation ([MORWiki thermal block](https://morwiki.mpi-magdeburg.mpg.de/morwiki/index.php/Thermal_Block))

<table>
<tr>
<td>

For $t \in (0, T)$, $\xi \in \Omega$ and initial values

$$
\theta(0, \xi) = 0,\text{ for } \xi \in \Omega,
$$

consider

$$
\begin{align*}
  \partial_t \theta(t, \xi)
  + \nabla \cdot (-\sigma(\xi) \nabla \theta(t, \xi))
  & = 0,
\end{align*}
$$

with boundary conditions

$$
\begin{align*}
  \sigma(\xi) \nabla \theta(t, \xi) \cdot n(\xi) & = u(t)
  & t \in (0, T),
  & \ \xi \in \Gamma_{\text{in}}, \\
  \sigma(\xi) \nabla \theta(t, \xi) \cdot n(\xi) & = 0
  & t \in (0, T),
  & \ \xi \in \Gamma_{\text{N}}, \\
  \theta(t, \xi) & = 0
  & t \in (0, T),
  & \ \xi \in \Gamma_{\text{D}},
\end{align*}
$$

and outputs

$$
y_i(t) = \int_{\Omega_i} \theta(t, \xi) \dif{\xi}, \quad
i = 1, 2, 3, 4.
$$

</td>
<td>
<center>
<img src="figures/cookie.svg" alt="cookie domain" width="30%">
<img src="figures/Euler_100_Tf.png" alt="cookie snapshot" width="30%">
</center>
</td>
</tr>
</table>

#### Finite element semi-discretization in space

- pairwise inner products of ansatz functions $\leadsto E$
- discretized spatial operator + Dirichlet boundary condition $\leadsto A$
- discretized non-zero Neumann boundary condition $\leadsto B$
- average temperatures on the inclusions $\leadsto C$

---

- $n = 7\,488$
- $m = 1$
- $p = 4$

### Penzl Example ([MORWiki](https://morwiki.mpi-magdeburg.mpg.de/morwiki/index.php/Penzl%27s_FOM))

In [ ]:
import numpy as np
import scipy.sparse as sps
from pymor.models.iosys import LTIModel

A1 = np.array([[-1, 100], [-100, -1]])
A2 = np.array([[-1, 200], [-200, -1]])
A3 = np.array([[-1, 400], [-400, -1]])
A4 = sps.diags(np.arange(-1, -1001, -1))
A = sps.block_diag((A1, A2, A3, A4), format='csc')
B = np.ones((1006, 1))
B[:6] = 10
C = B.T

fom = LTIModel.from_matrices(A, B, C)

In [ ]:
fom

In [ ]:
print(fom)

We can perform time-domain simulation, but the final time and the time stepper need to be specified in the `Model`.

In [ ]:
from pymor.algorithms.timestepping import ImplicitEulerTimeStepper

T = 2
nt = 10000
fom = fom.with_(T=T, time_stepper=ImplicitEulerTimeStepper(nt))

We first simulate the impulse response, i.e., the output in response to $x(0) = 0$ and $u(t) = \delta(t)$.

$$
\begin{align*}
  y(t)
  & =
    C e^{t E^{-1} A} x(0)
    + \int_0^t C e^{\tau E^{-1} A} E^{-1} B u(t - \tau) \operatorname{d\!}\tau
    + D u(t) \\
  & =
    C e^{t E^{-1} A} E^{-1} B
    + D \delta(t)
\end{align*}
$$

In [ ]:
y_impulse = fom.impulse_resp()
print(y_impulse.shape)

In [ ]:
import matplotlib.pyplot as plt

_ = plt.plot(np.linspace(0, T, nt + 1), y_impulse[:, 0, 0])

Next we simulate the response to a sinusoidal input.

In [ ]:
y_sin_100 = fom.output(input='sin(100 * t)')
print(y_sin_100.shape)

In [ ]:
_ = plt.plot(np.linspace(0, T, nt + 1), y_sin_100[:, 0])

In [ ]:
y_sin_50 = fom.output(input='sin(50 * t)')

In [ ]:
_ = plt.plot(np.linspace(0, T, nt + 1), y_sin_50[:, 0])

#### Exercise

- Use the time stepper specified above and simulate the model using the input function $u(t) = e^{-t}$. 
- Change the number of timesteps `nt` to `50000`. Repeat the simulation of the model using $u(t) = e^{-t}$.

# Transfer Function

## Laplace Transform

> ### Definition
>
> Let $f \colon [0, \infty) \to \Rn$ be exponentially bounded with bounding
> exponent $\alpha$.
> Then
> $$\cL\{f\}(s) := \int_0^\infty f(\tau) e^{-s \tau} \dif{\tau}$$
> for $\Real(s) > \alpha$ is called the ***Laplace transform*** of $f$.
> The process of forming the Laplace transform is called
> ***Laplace transformation***.

It can be shown that the integral converges uniformly in a domain with
$\Real(s) \ge \beta$ for all $\beta > \alpha$.

> Allows us to map time signals to frequency signals.

> ### Theorem
>
> Let $f, g, h \colon [0, \infty) \to \bbR^n$ be given.
> Then the following two statements hold true:
>
> 1. The Laplace transformation is linear, i.e.,
>    if $f$ and $g$ are exponentially bounded,
>    then $h := \gamma f + \delta g$ is also exponentially bounded and
>
>    $$
     \cL\left\{h\right\} = \gamma\cL\left\{f\right\} +
     \delta\cL\left\{g\right\}
     $$
>
>    holds for all $\gamma, \delta \in \bbC$.
> 2. If $f \in \mathcal{PC}^1([0, \infty), \Rn)$ and $\dot{f}$ is exponentially
>    bounded, then $f$ is exponentially bounded and
>
>    $$
     \cL\bigl\{\dot{f}\bigr\}(s) = s \cL\{f\}(s) - f(0).
     $$

- $X(s) := \cL\{x\}(s)$,
  $U(s) := \cL\{u\}(s)$, and
  $Y(s) := \cL\{y\}(s)$
- $A x(t) + B u(t) \leadsto A X(s) + B U(s)$
- $y(t) = C x(t) \leadsto Y(s) = C X(s)$
- $s X(s) := \cL\{\dot{x}\}(s)$ (since $x(0) = 0$)

## Transfer Function

In summary we have:

- $s E X(s) = A X(s) + B U(s)$
- $Y(s) = C X(s)$

Thus the mapping from inputs to outputs in frequency domain can be expressed as

$$
H(s) = C {\left(s E - A\right)}^{-1} B.
$$

$$
H \text{ is analytic in } \bbC \setminus \Lambda(E, A).
$$

### Pole-residue Form

Let $(\lambda_{i}, w_{i}, v_{i})$ be the eigentriplets of the pair $(E, A)$
with no degenerate eigenspaces. Then the ***poles*** of $H$ are given by the eigenvalues $\lambda_1,\ldots,\lambda_n$ and we have

$$
H(s) = \sum_{i = 1}^{n} \frac{R_{i}}{s - \lambda_{i}},
$$

where $R_{i} = (C v_{i}) (w_{i}^{\herm} B)$,
assuming $w_{i}^{\herm} E v_{i} = 1$.

### Example

In [ ]:
fom.transfer_function

In [ ]:
fom.transfer_function.eval_tf(0)

In [ ]:
fom.transfer_function.eval_tf(10j)

#### Exercise

- Use the [`poles`](https://docs.pymor.org/2023-1-0/autoapi/pymor/models/iosys/index.html#pymor.models.iosys.LTIModel.poles) method of the [`LTIModel`](https://docs.pymor.org/2023-1-0/autoapi/pymor/models/iosys/index.html#pymor.models.iosys.LTIModel) class to compute the poles of the transfer function of `fom`. Compute the imaginary parts of the poles.
- Evaluate the transfer function for several values on the imaginary axis. Select some values that correspond to the imaginary parts of the poles and others which are close by.

### Frequency-Domain Analysis

#### Bode Plots

The Bode plot for $H$ consists of a ***magnitude plot*** and a ***phase plot***.

> ##### Bode magnitude plot
>
> - component-wise graph of the function $\lvert H(\imag \omega) \rvert$
>   for frequencies $\omega \in [\omega_{\min}, \omega_{\max}] \subset \bbR$.
> - $\omega$-axis is logarithmic.
> - magnitude is given in decibels, i.e., $\lvert H(\imag \cdot) \rvert$ is
>   plotted as $20 \log_{10}(\lvert H(\imag \cdot) \rvert)$.

> ##### Bode phase plot
>
> - component-wise graph of the function $\arg{H(\imag \omega)}$
>   for frequencies $\omega \in [\omega_{\min}, \omega_{\max}] \subset \bbR$.
> - $\omega$-axis is logarithmic.
> - phase is given in degrees on a linear scale.

#### Bode Plot for the Thermal Block Example

<center>
<img src="figures/cookie_bode.svg" alt="cookie bode" width="60%">
</center>

In [ ]:
# w = (1e-1, 1e5)
w, _ = fom.transfer_function.freq_resp((1e-1, 1e5))

In [ ]:
_ = fom.transfer_function.bode_plot(w)

> #### (Sigma) Magnitude Plots
>
> - 2-norm-wise graph of the function $H(\imag \omega)$
>   for frequencies $\omega \in [\omega_{\min}, \omega_{\max}] \subset \bbR$.
> - $\omega$-axis is logarithmic.

The name is due to the fact that for a given matrix $M$ the norm
$\lVert M \rVert_2$ is given by its largest singular value.

The real sigma magnitude plot depicts all singular values as functions of
$\omega$.

In [ ]:
_ = fom.transfer_function.mag_plot(w)

## Projection-based MOR

### Ritz/Petrov-Galerkin Projection

$$
\begin{align*}
  E \dot{x}(t) - A x(t) - B u(t) & = 0, \\
  y(t) - C x(t) - D u(t) & = 0.
\end{align*}
$$

**Step I: Use truncated state transformation**

Replace

$$
x(t) \approx V \hx(t)
$$

with $V \in \Rnr$ and $\hx(t) \in \Rr$.

$$
\begin{align*}
  E V \dot{\hx}(t) - A V \hx(t) - B u(t) & = e_{\text{res}}(t), \\
  y(t) - C V \hx(t) - D u(t) & = e_{\text{output}}(t).
\end{align*}
$$

**Step II: Mitigate transformation error**

Suppress truncation residual through left projection.

- one-sided method: use $V$ again.

  $$
  \begin{align*}
    V^{\tran} E V \dot{\hx}(t)
    - V^{\tran} A V \hx(t)
    - V^{\tran} B u(t)
    & = 0, \\
    y(t) - C V \hx(t) - D u(t) & = e_{\text{output}}(t).
  \end{align*}
  $$

- two-sided method: find $W \in \Rnr$.

  $$
  \begin{align*}
    W^{\tran} E V \dot{\hx}(t)
    - W^{\tran} A V \hx(t)
    - W^{\tran} B u(t)
    & = 0, \\
    y(t) - C V \hx(t) - D u(t) & = e_{\text{output}}(t).
  \end{align*}
  $$

<center>
<img src="figures/compress_A.svg" alt="compress A" width="80%">
</center>

### Reduced order model (ROM) (pyMOR: [`LTIPGReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/basic/index.html?highlight=ltipgred#pymor.reductors.basic.LTIPGReductor))

Define
$\hE = W^{\tran} E V$,
$\hA = W^{\tran} A V \in \Rrr$,
$\hB = W^{\tran} B \in \Rrm$, and
$\hC = C V \in \Rpr$.
Then

$$
\begin{equation}\tag{ROM}
  \begin{aligned}
    \hE \dot{\hx}(t) & = \hA \hx(t) + \hB u(t), \\
    \hy(t) & = \hC \hx(t) + D u(t)
  \end{aligned}
\end{equation}
$$

approximates the dynamics of the full-order model $\Sigma$ with output error

$$
y(t) - \hy(t) = e_{\text{output}}(t).
$$

- We call the corresponding transfer function $\hH$.
- Model order reduction (MOR) $\leadsto$
  Find $W, V \in \Rnr$ such that $e_{\text{output}}(t)$ is small in a suitable
  sense.
- We will focus on eigenvalue-based, energy-based and
  interpolation-based methods today.

### Example

In [ ]:
from pymor.reductors.basic import LTIPGReductor

V = fom.solution_space.random(10)
pg = LTIPGReductor(fom, V, V)
rom_pg = pg.reduce()

The resulting model `rom_pg` will again be an [`LTIModel`](https://docs.pymor.org/2023-1-0/autoapi/pymor/models/iosys/index.html?highlight=ltimodel#pymor.models.iosys.LTIModel).

In [ ]:
rom_pg

In [ ]:
_ = fom.transfer_function.mag_plot(w, label='FOM')
_ = rom_pg.transfer_function.mag_plot(w, label='Random PG')
_ = plt.legend()

#### Exercise

- Simulate the state of the `rom_pg` with $u(t) = e^{-t}$ from $t_{start}=0$ to $t_{end}=2$ using its [`solve`](https://docs.pymor.org/2023-1-0/autoapi/pymor/models/interface/index.html#pymor.models.interface.Model.solve) method.
- The solve method computes a [`VectorArray`](https://docs.pymor.org/2023-1-0/autoapi/pymor/vectorarrays/interface/index.html?highlight=vectorarray#pymor.vectorarrays.interface.VectorArray) with all computed state vectors from the time-domain simulation. Consider the last state (i.e., at time $t_{end}$) from the previous computation. Use the [`reconstruct`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/basic/index.html?highlight=reconstruct#pymor.reductors.basic.LTIPGReductor.reconstruct) method of `pg` to obtain the reconstructed state vector in the full-order model state space.
- Perform the same simulation with `fom`. Compare the final state with the state that has been reconstructed using the ROM simulation.

# System Analysis

## System Norms and Hardy Spaces

We have $$Y(s) = H(s) U(s)$$ and $$\hY(s) = \hH(s) U(s).$$

> ### Question
>
> What are suitable norms such that
>
> $$
  \lVert y - \hy \rVert
  \le
  \left\lVert H - \hH \right\rVert
  \lVert u \rVert?
  $$

### The Banach Space $\cH_\infty^{p \times m}$

$$
\cH_\infty^{p \times m}
:=
\left\{
  G \colon \bbC^+ \to \Cpm :
  G \text{ is analytic in $\bbC^+$ and }
  \sup_{s \in \bbC^+} \left\lVert G(s) \right\rVert_2 < \infty
\right\}.
$$

$\cH_\infty^{p \times m}$ is a Banach space equipped with the
***$\cH_\infty$-norm***

$$
\left\lVert G \right\rVert_{\cH_\infty}
:= \sup_{\omega \in \bbR} \left\lVert G(\imag \omega) \right\rVert_2.
$$

> Can show:
>
> $$
  \lVert y - \hy \rVert_{\cL_{2}}
  \le
  \left\lVert H - \hH \right\rVert_{\cH_{\infty}}
  \lVert u \rVert_{\cL_{2}}.
  $$

This bound can even be shown to be sharp.

In [ ]:
fom.hinf_norm()

### The Hilbert Space $\cH_2^{p \times m}$

$$
\cH_2^{p \times m}
:= \left\{
  G \colon \bbC^+ \to \Cpm :
  G \text{ is analytic in $\bbC^+$ and }
  \sup_{\xi > 0}
  \int_{-\infty}^\infty
  \left\lVert
  G(\xi + \imag \omega)
  \right\rVert_{\operatorname{F}}^2
  \dif{\omega}
  < \infty
\right\}.
$$

$\cH_2^{p \times m}$ is a Hilbert space with the inner product

$$
\langle F, G \rangle_{\cH_2}
:=
\frac{1}{2 \pi}
\int_{-\infty}^\infty
\tr\!\left({F(\imag \omega)}^{\herm} G(\imag \omega)\right)
\dif{\omega}
$$

and induced norm

$$
\left\lVert G \right\rVert_{\cH_2}
:= \langle G, G \rangle_{\cH_2}^{1/2}
= {
  \left(
    \frac{1}{2 \pi}
    \int_{-\infty}^\infty
    \left\lVert G(\imag \omega) \right\rVert_{\operatorname{F}}^2
    \dif{\omega}
  \right)
}^{1/2}.
$$

> Can show:
>
> $$
  \lVert y - \hy \rVert_{\cL_{\infty}}
  \le
  \left\lVert H - \hH \right\rVert_{\cH_{2}}
  \lVert u \rVert_{\cL_{2}}.
  $$

In [ ]:
fom.h2_norm()

### System Gramians and $\cH_{2}$ trace formula

A system $\Sigma$ with $\Lambda(E, A) \subset \bbC^{-}$ is called
***asymptotically stable***.
Then, all state trajectories decay exponentially as $t \to \infty$ and

- the infinite controllability and observability ***Gramians*** exist:

  $$
  \begin{align*}
    P
    & =
      \int_0^{\infty}
      e^{E^{-1} A t}
      E^{-1}
      B B^{\tran}
      E^{-\tran}
      e^{A^{\tran} E^{-\tran} t}
      \dif{t} \\
    E^{\tran} Q E
    & =
      \int_0^{\infty}
      e^{A^{\tran} E^{-\tran} t}
      C^{\tran} C
      e^{E^{-1} A t}
      \dif{t}.
  \end{align*}
  $$
- $P$, $Q$ solve the two ***Lyapunov equations***

  $$
  A P E^{\tran} + E P A^{\tran} + B B^{\tran} = 0, \qquad
  A^{\tran} Q E + E^{\tran} Q A + C^{\tran} C = 0
  $$
<!-- - If $(A, B)$ is controllable and $(A, C)$ is observable, -->
<!--   it moreover holds that $P = P^{\tran} \succ 0$ and $Q = Q^{\tran} \succ 0$. -->
<!--   (Otherwise we just have $P = P^{\tran} \succcurlyeq 0$ and -->
<!--   $Q = Q^{\tran} \succcurlyeq 0$.) -->
- the $\cH_{2}$-norm can be expressed as

  $$
  \lVert H \rVert_{\cH_{2}}^{2}
  = \tr\!\left(C P C^{\tran}\right)
  = \tr\!\left(B^{\tran} Q B\right).
  $$

# A Selection of MOR Methods

## Modal Methods

### Modal Coordinates

Assume that the pair $(E, A)$ is simultaneously diagonalizable in $\Cnn$.

> #### Classic Modal Truncation
>
> - Compute diagonal realization from an eigendecomposition.
> - State-space transformation matrices contain eigenvectors (modes).
> - Use $W = V$.
> - Populate $V$ with modes corresponding to eigenvalues closest to
>   $\imag \bbR$.
> - Add a few domain-specific or "anxiety" modes.

> #### Problem
>
> - Does not take inputs and outputs into account!
> - How many "anxiety" modes are necessary?

### Dominant Poles Approximation (pyMOR: [`MTReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/mt/index.html?highlight=mtreductor#pymor.reductors.mt.MTReductor))

Recall the pole residue form of the transfer function

$$
H(s) = \sum_{i = 1}^{n} \frac{R_{i}}{s - \lambda_{i}},
$$

where $R_{i} = (C v_{i})(w_{i}^{\herm} B)$, assuming
$w_{i}^{\herm} E v_{i} = 1$.

Suppose the modes are sorted based on the magnitude of the
$\lVert R_{i} \rVert / \Real(\lambda_{i})$.
Then we use the truncated pole residue form

$$
H(s) = \sum_{i = 1}^{r} \frac{R_{i}}{s - \lambda_{i}},
$$

as our ROM. This is motivated by the following

> #### Error bound
>
> $$
  \left\lVert H - \hH \right\rVert_{\cH_\infty}
  \le
  \sum_{i = r + 1}^{n}
  \frac{\lVert R_{i} \rVert}{\lvert \Real(\lambda_{i}) \rvert}
  $$

Computation is feasible via *subspace accelerated MIMO dominant pole algorithm*
(SAMDP).

In [ ]:
from pymor.reductors.mt import MTReductor

mt = MTReductor(fom)
rom_mt = mt.reduce(10)

In [ ]:
_ = fom.transfer_function.mag_plot(w, label='FOM')
_ = rom_mt.transfer_function.mag_plot(w, label='MT')
_ = plt.legend()

In [ ]:
err_mt = fom - rom_mt

In [ ]:
_ = err_mt.transfer_function.mag_plot(w, label='MT')
_ = plt.legend()

#### Exercise

- Compute the $\mathcal{H}_2$-norm of the error system `err_mt`. Note that the [`LTIModel`](https://docs.pymor.org/2023-1-0/autoapi/pymor/models/iosys/index.html?highlight=ltimodel#pymor.models.iosys.LTIModel) class has an [`h2_norm`](iosys/index.html?highlight=ltimodel#pymor.models.iosys.LTIModel.h2_norm) method.
- Consider the input $u(t) = e^{-t}$ which has the $\cL_{2}$-norm $\lVert u \rVert_{\cL_{2}} = \frac{\sqrt{2}}{2}$. Simulate the error system `err_mt` with the input $u(t) = e^{-t}$ and verify the input-output error bound $\lVert y - \hy \rVert_{\cL_{\infty}}
  \le
  \left\lVert H - \hH \right\rVert_{\cH_{2}}
  \lVert u \rVert_{\cL_{2}}$.

## Balancing-based MOR

### Balanced Truncation aka. Lyapunov Balancing

#### Idea

- The system $\Sigma$, in realization $(E = I, A, B, C)$,
  is called ***balanced***, if the solutions $P, Q$ of the Lyapunov equations

  $$
  A P + P A^{\tran} + B B^{\tran} = 0, \qquad
  A^{\tran} Q + Q A + C^{\tran} C = 0,
  $$

  satisfy:
  $P = Q = \operatorname{diag}(\sigma_1, \ldots, \sigma_n)$
  where
  $\sigma_1 \ge \sigma_2 \ge \cdots \ge \sigma_n > 0$.

- $\{\sigma_1, \ldots, \sigma_n\}$ are the *Hankel singular values (HSVs)* of
  $\Sigma$.

- A so-called ***balanced realization*** is computed via state-space transformation

  $$
  \begin{align*}
    \cT \colon (I, A, B, C) \mapsto {} & (I, T A T^{-1}, T B, C T^{-1}) \\
    & =
      \left(
        I,
        \begin{bmatrix}
          A_{11} & A_{12} \\
          A_{21} & A_{22}
        \end{bmatrix},
        \begin{bmatrix}
          B_{1} \\
          B_{2}
        \end{bmatrix},
        \begin{bmatrix}
          C_{1} & C_{2}
        \end{bmatrix}
      \right).
  \end{align*}
  $$

- In a balanced realization the state variables are sorted based on their contribution to the input-output mapping.

- Truncation removes state variables which are not important for input-ouput behaviour $\leadsto$ reduced order model:
  $(I, \hA, \hB, \hC) = (I, A_{11}, B_{1}, C_{1})$.

### Implementation: The Square Root Method

#### The SR Method (pyMOR: [`BTReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/bt/index.html?highlight=btreductor#pymor.reductors.bt.BTReductor))

1. Compute (Cholesky) factors of the solutions to the Lyapunov equation,

   $$
   P = S^{\tran} S, \quad
   Q = R^{\tran} R.
   $$

2. Compute singular value decomposition

   $$
   S R^{\tran}
   =
   \begin{bmatrix}
     U_1 & U_2
   \end{bmatrix}
   \begin{bmatrix}
     \Sigma_1 & 0 \\
     0 & \Sigma_2
   \end{bmatrix}
   \begin{bmatrix}
     V_1^{\tran} \\
     V_2^{\tran}
   \end{bmatrix}.
   $$

3. Define

   $$
   W := R^{\tran} V_1 \Sigma_1^{-1/2}, \quad
   V := S^{\tran} U_1 \Sigma_1^{-1/2}.
   $$
4. Then the reduced-order model is
   $(W^{\tran} A V, W^{\tran} B, C V)$.

#### Properties

- Lyapunov balancing **preserves asymptotic stability**.
- We have the **a priori error bound**:
  $$
  \left\lVert H - \hH \right\rVert_{\cH_{\infty}}
  \le
  2 \sum\limits_{k = r + 1}^{n} \sigma_{k}
  $$

#### Variants (pyMOR: [`PRBTReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/bt/index.html?highlight=btreductor#pymor.reductors.bt.PRBTReductor), [`BRBTReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/bt/index.html?highlight=btreductor#pymor.reductors.bt.BRBTReductor), [`LQGBTReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/bt/index.html?highlight=btreductor#pymor.reductors.bt.LQGBTReductor))

Other versions for special classes of systems or applications exist, such as

- **positive-real balancing** (passivity-preserving),
- **bounded-real balancing** (contractivity-preserving),
- **linear-quadratic Gaussian balancing**
  (stability preserving, aims at low-order output feedback controllers).

The given ones all compute $P, Q$ as solutions of ***algebraic Riccati
equations*** of the form:

$$
\begin{align*}
  0
  & =
    \tA P \tE^{\tran}
    + \tE P \tA^{\tran}
    + \tB \tB^{\tran}
    \pm \tE P \tC^{\tran} \tC P \tE^{\tran} \\
  0
  & =
    \tA^{\tran} Q \tE
    + \tE^{\tran} Q \tA
    + \tC^{\tran} \tC
    \pm \tE^{\tran} Q \tB \tB^{\tran} Q \tE.
\end{align*}
$$

In [ ]:
from pymor.reductors.bt import BTReductor

bt = BTReductor(fom)
rom_bt = bt.reduce(10)

In [ ]:
_ = fom.transfer_function.mag_plot(w, label='FOM')
_ = rom_mt.transfer_function.mag_plot(w, label='MT')
_ = rom_bt.transfer_function.mag_plot(w, label='BT')
_ = plt.legend()

In [ ]:
err_bt = fom - rom_bt

In [ ]:
_ = err_mt.transfer_function.mag_plot(w, label='MT')
_ = err_bt.transfer_function.mag_plot(w, label='BT')
_ = plt.legend()

#### Exercise

- Aside from a desired order, the [`reduce`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/bt/index.html?highlight=btreductor#pymor.reductors.bt.GenericBTReductor.reduce) method of the [`BTReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/bt/index.html?highlight=btreductor#pymor.reductors.bt.BTReductor) allows for specifying a truncation tolerance based on the a priori error bound. Use the `bt` instance of the [`BTReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/bt/index.html?highlight=btreductor#pymor.reductors.bt.BTReductor) to compute a ROM based on a specified tolerance `tol=1e-5`.
- Use the [`LQGBTReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/bt/index.html?highlight=btreductor#pymor.reductors.bt.LQGBTReductor) to reduce `fom` using a truncation tolerance of `tol=1e-5`. Check the dimension of the ROM.
- Compare the $\cH_{2}$-norms and orders of the ROMs obtained by both BT variants.

## Transfer Function Approximation

The transfer function $H$ is a degree-$n$ rational function

$$
H(s) = \frac{P(s)}{Q(s)}, \quad P,Q \text{ polynomials}
$$

such that deg$(P) \leq n$ and deg$(Q) \leq n$.

MOR via rational approximation: Find a degree-$r$ rational function $\hH$ with $r \ll n$ such that

$$
H \approx \hH
$$

### Rational Interpolation

Pick some complex values $\sigma_1, \ldots, \sigma_r$ and enforce interpolation

$$
\hH(\sigma_j) = H(\sigma_j) \quad \text{for } j = 1, \ldots, r.
$$

Can also pick tangential directions $b_1, \ldots, b_r \in \mathbb{C}^m$ and $c_1, \ldots, c_r \in \mathbb{C}^p$ and enforce bitangential Hermite interpolation

$$
\begin{align*}
  \hH(\sigma_j)b_j &= H(\sigma_j) b_j, \\
  c_j^* \hH(\sigma_j) &= c_j^* H(\sigma_j), \\
  c_j^* \hH'(\sigma_j) b_j &= c_j^* H'(\sigma_j) b_j, \\
\end{align*}
\quad \text{for } j=1,\ldots,r.
$$

### Interpolation via Projection

Given $E,A,B,C$, how to enforce interpolation?

> ### Theorem
>
> Let $\hH$ be the transfer function of the ROM obtained from a Petrov Galerkin projection using $V$ and $W$.
> For $b \in \mathbb{C}^m$, $c \in \mathbb{C}^p$ and $\sigma \in \mathbb{C}$ we have
>
> 1. $(\sigma E - A)^{-1} B b \in \mathrm{Range}(V)$ implies $H(\sigma) b = \hH(\sigma) b$
> 2. $(\sigma E - A)^{-*} C^* c \in \mathrm{Range}(W)$ implies $c^* H(\sigma) = c^* \hH(\sigma)$
> 3. If 1. and 2. are satisfied, then $c^* H'(\sigma) b = c^* \hH'(\sigma) b$

Using bases $V$ and $W$ for rational Krylov subspaces allows for interpolatory MOR [[Antoulas/Beattie/Güğercin '20]](https://doi.org/10.1137/1.9781611976083).

In pyMOR [`LTIBHIReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/interpolation/index.html?highlight=ltibhired#pymor.reductors.interpolation.LTIBHIReductor) is based on projection and [`TFBHIReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/interpolation/index.html?highlight=tfbhi#pymor.reductors.interpolation.TFBHIReductor) only uses evaluations of the transfer function $H$.

In [ ]:
from pymor.reductors.interpolation import LTIBHIReductor

interp = LTIBHIReductor(fom)
sigma = np.array([50, 100, 200, 400, 800])
sigma = np.concatenate((1j * sigma, -1j * sigma))
b = np.ones((len(sigma), fom.dim_input))
c = np.ones((len(sigma), fom.dim_output))
rom_interp = interp.reduce(sigma, b, c)

In [ ]:
_ = fom.transfer_function.mag_plot(w, label='FOM')
_ = rom_mt.transfer_function.mag_plot(w, label='MT')
_ = rom_bt.transfer_function.mag_plot(w, label='BT')
_ = rom_interp.transfer_function.mag_plot(w, label='Interpolation')
_ = plt.legend()

In [ ]:
err_interp = fom - rom_interp

In [ ]:
_ = err_mt.transfer_function.mag_plot(w, label='MT')
_ = err_bt.transfer_function.mag_plot(w, label='BT')
_ = err_interp.transfer_function.mag_plot(w, label='Interpolation')
_ = plt.legend()

### Iterative Rational Krylov Algorithm (IRKA)

> #### $\cH_2$-optimal MOR problem
>
> Find a stable $\hH$ of order $r$ such that $\lVert H - \hH \rVert_{\cH_{2}}$
> is minimized.

> #### Interpolatory necessary $\cH_2$-optimality conditions
>
> Let $\hH(s) = \sum_{i = 1}^r \frac{\phi_i}{s - \lambda_i}$ be an
> $\cH_2$-optimal reduced-order model for $H$.
> Then
> $$
  \begin{align*}
    H\!\left(-\overline{\lambda_i}\right)
    & = \hH\!\left(-\overline{\lambda_i}\right), \\
    H'\!\left(-\overline{\lambda_i}\right)
    & = \hH'\!\left(-\overline{\lambda_i}\right),
  \end{align*}
  $$
> for $i = 1, 2, \ldots, r$.

> ***Hermite interpolation*** of the transfer function is necessary for $\cH_2$-optimality.

> #### IRKA (pyMOR: [`IRKAReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/h2/index.html?highlight=irkared#pymor.reductors.h2.IRKAReductor))
>
> Fixed point iteration based on interpolatory necessary optimality conditions.

In [ ]:
from pymor.reductors.h2 import IRKAReductor

irka = IRKAReductor(fom)
rom_irka = irka.reduce(10)

In [ ]:
_ = fom.transfer_function.mag_plot(w, label='FOM')
_ = rom_mt.transfer_function.mag_plot(w, label='MT')
_ = rom_bt.transfer_function.mag_plot(w, label='BT')
_ = rom_interp.transfer_function.mag_plot(w, label='Interpolation')
_ = rom_irka.transfer_function.mag_plot(w, label='IRKA')
_ = plt.legend()

In [ ]:
err_irka = fom - rom_irka

In [ ]:
_ = err_mt.transfer_function.mag_plot(w, label='MT')
_ = err_bt.transfer_function.mag_plot(w, label='BT')
_ = err_interp.transfer_function.mag_plot(w, label='Interpolation')
_ = err_irka.transfer_function.mag_plot(w, label='IRKA')
_ = plt.legend()

#### Exercise

- Compute 5 random values `sigma` in the interval $[0,1000]$ using `np.random.rand`. Then use the [`TFBHIReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/interpolation/index.html?highlight=tfbhi#pymor.reductors.interpolation.TFBHIReductor) to compute a ROM which interpolates `fom.transfer_function` at the random values and its complex conjugates along the imaginary axis (i.e., use interpolation points `np.concatenate((1j * sigma, -1j * sigma))`). Compute the relative $\cH_{2}$-error of the resulting ROM. Compare it to the relative $\cH_{2}$-error of the interpolation points chosen in the example above where we chose `sigma = np.array([50, 100, 200, 400, 800])`.
- Repeat the previous computations with 10 random values (i.e., a total of 20 interpolation points).
- Use the `sigma` specified below as an initial guess for the [`reduce`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/h2/index.html?highlight=irkared#pymor.reductors.h2.IRKAReductor.reduce) method of the [`IRKAReductor`](https://docs.pymor.org/2023-1-0/autoapi/pymor/reductors/h2/index.html?highlight=irkared#pymor.reductors.h2.IRKAReductor):
```
sigma = np.array([50, 100, 200, 400, 800]);
sigma = np.concatenate((1j * sigma, -1j * sigma))
```

<center>Questions?</center>